In [ ]:
'''
Author: miyeonss (https://github.com/miyeonss)
'''

In [1]:
from gym import Env #customer env
from gym.spaces import Discrete, Box, Tuple, Dict, MultiBinary, MultiDiscrete  #can define action, current state about env
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [2]:
class CounterEnv_DQN(Env):
    def __init__(self):
        #actions increment, decrement, consistent
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high = np.array([10]))
        #set start number
        self.state = 0 + random.randint(-2,2)
        #set counter length
        self.counter_length = 10
        
    def step(self, action):
        #apply action
        #0 -1 = -1 -> decrement
        #1 -1 = 0 -> consistent
        #2 -1 = 1 -> increment
        self.state += action -1
        #reduce counter length by 1 time
        self.counter_length -= 1
        
        #calculate reward
        if self.state >= 8:
            reward = 1
        else:
            reward = -1
            
        #check if counter is done
        if self.counter_length <= 0:
            done = True
        else: 
            done = False
        
        #noise
        self.state += random.randint(-1,1)
        
        info = {}
        
        #return step information
        return self.state, reward, done, info
    
    def render(self):
        pass
    
    def reset(self):
        #reset counter
        self.state = 0 + random.randint(-2,2)
        #reset counter time
        self.counter_length = 10
        return self.state

In [3]:
env = CounterEnv_DQN()

/home/kimnj3050/.conda/envs/njenv/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
env.observation_space.sample()

array([0.88056505], dtype=float32)

In [5]:
env.action_space.sample()

2

In [6]:
episodes = 20
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{} reward:{} '.format(episode, score, reward))
    


Episode:1 Score:-10 reward:-1 
Episode:2 Score:-10 reward:-1 
Episode:3 Score:-10 reward:-1 
Episode:4 Score:-10 reward:-1 
Episode:5 Score:-10 reward:-1 
Episode:6 Score:-10 reward:-1 
Episode:7 Score:-10 reward:-1 
Episode:8 Score:-10 reward:-1 
Episode:9 Score:-10 reward:-1 
Episode:10 Score:-4 reward:1 
Episode:11 Score:-10 reward:-1 
Episode:12 Score:-10 reward:-1 
Episode:13 Score:-10 reward:-1 
Episode:14 Score:-2 reward:1 
Episode:15 Score:-10 reward:-1 
Episode:16 Score:-10 reward:-1 
Episode:17 Score:-10 reward:-1 
Episode:18 Score:-10 reward:-1 
Episode:19 Score:-10 reward:-1 
Episode:20 Score:-6 reward:1 


In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [8]:
states = env.observation_space.shape
actions = env.action_space.n

In [9]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation = 'relu', input_shape = states))
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(action, activation ='linear'))
    return model

In [10]:
model = build_model(states, actions)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 698
Trainable params: 698
Non-trainable params: 0
_________________________________________________________________


In [12]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory